# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import pandas as pd
import os
from skimage.measure._regionprops import RegionProperties
from towbintools.foundation.image_handling import read_tiff_file
from skimage.measure import label, regionprops_table
from towbintools.foundation.image_handling import pad_images_to_same_dim
from sklearn.model_selection import train_test_split
import yaml
from tqdm import tqdm
from csbdeep.utils import normalize

def get_all_skimage_regionprops():

    features = [attr for attr in dir(RegionProperties)
                if not attr.startswith('_') and
                isinstance(getattr(RegionProperties, attr), property) if "image" not in attr and "coords" not in attr]
    return features

features = get_all_skimage_regionprops()
print("Skimage Region Properties Features:")
for feature in features:
    print(feature)

Skimage Region Properties Features:
area
area_bbox
area_convex
area_filled
axis_major_length
axis_minor_length
bbox
centroid
centroid_local
centroid_weighted
centroid_weighted_local
eccentricity
equivalent_diameter_area
euler_number
extent
feret_diameter_max
inertia_tensor
inertia_tensor_eigvals
intensity_max
intensity_mean
intensity_min
intensity_std
moments
moments_central
moments_hu
moments_normalized
moments_weighted
moments_weighted_central
moments_weighted_hu
moments_weighted_normalized
num_pixels
orientation
perimeter
perimeter_crofton
solidity


In [2]:
dataset_path = "/mnt/towbin.data/shared/spsalmon/towbinlab_classification_database/datasets/10x_body_qc/body"
project_yaml = os.path.join(dataset_path, "project.yaml")
with open(project_yaml, 'r') as f:
    project_config = yaml.safe_load(f)

classes = project_config['classes']
print("Classes in the dataset:", classes)
image_dir = os.path.join(dataset_path, "images")
mask_dir = os.path.join(dataset_path, "masks")
annotation_csv_path = os.path.join(dataset_path, "annotations", "annotations.csv")
annotations_df = pd.read_csv(annotation_csv_path)
# replace //izbkingston with /mnt
annotations_df['ImagePath'] = annotations_df['ImagePath'].str.replace('//izbkingston', '/mnt')
# replace \ with /
annotations_df['ImagePath'] = annotations_df['ImagePath'].str.replace('\\', '/')
# add the mask paths to the dataframe
annotations_df['MaskPath'] = annotations_df['ImagePath'].apply(lambda x: os.path.join(mask_dir, os.path.basename(x)))

# check that image and mask paths exist
for idx, row in annotations_df.iterrows():
    assert os.path.exists(row['ImagePath']), f"Image path does not exist: {row['ImagePath']}"
    assert os.path.exists(row['MaskPath']), f"Mask path does not exist: {row['MaskPath']}"

Classes in the dataset: ['egg', 'good', 'bad', 'unusable']


In [3]:
print(annotations_df.shape)

(1999, 3)


# xgboost

In [4]:
# extract features and labels
from towbintools.foundation.image_handling import read_tiff_file
X = []
for idx, row in tqdm(annotations_df.iterrows()):
    try:
        image = read_tiff_file(row['ImagePath'])

        mask = read_tiff_file(row['MaskPath'])

        # if they are not the same shape, pad the smaller one with zeros
        if image.shape != mask.shape:
            image, mask = pad_images_to_same_dim(image, mask)

        # normalize image
        image = normalize(image, 1, 99.8, axis=None)
        # ensure mask is binary
        mask = (mask > 0).astype(np.uint8)
        props = regionprops_table(mask, intensity_image=image, properties=features)
        props_df = pd.DataFrame(props)
        if props_df.shape[0] == 0:
            # remove this row from annotations_df
            print(f"No regions found in mask for index {idx}, skipping.")
            annotations_df = annotations_df.drop(idx)
            continue
        X.append(props_df)

        # print(props_df.head())
    except Exception as e:
        print(f"Error processing index {idx}: {e}")

28it [00:00, 69.07it/s]/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
199it [00:02, 81.93it/s]/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
/home/spsalmon/micromamba/envs/towbintools/lib/

No regions found in mask for index 203, skipping.
No regions found in mask for index 204, skipping.
No regions found in mask for index 205, skipping.


/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regio

No regions found in mask for index 209, skipping.
No regions found in mask for index 210, skipping.
No regions found in mask for index 211, skipping.
No regions found in mask for index 212, skipping.
No regions found in mask for index 214, skipping.


/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
221it [00:03, 30.57it/s]

...

687it [00:42, 12.16it/s]/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-package

No regions found in mask for index 687, skipping.
No regions found in mask for index 688, skipping.
No regions found in mask for index 689, skipping.
No regions found in mask for index 690, skipping.
No regions found in mask for index 691, skipping.


694it [00:42, 13.38it/s]/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/home/spsalmon/micromamba/envs/towbintools/lib/python3.12/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (
698it [00:42, 17.13it/s]

No regions found in mask for index 695, skipping.
No regions found in mask for index 697, skipping.


1999it [01:41, 19.74it/s]


In [5]:
print(len(X), "feature dataframes extracted.")
features_df = pd.concat(X, ignore_index=True)
labels = annotations_df['Class'].values
# convert classes to integers
class_to_int = {cls: i for i, cls in enumerate(classes)}
labels = np.array([class_to_int[cls] for cls in labels])
print("Extracted Features DataFrame:")

print(features_df.shape)
print(labels.shape)

train_X, test_X, train_y, test_y = train_test_split(features_df, labels, test_size=0.2, random_state=42, stratify=labels)


1784 feature dataframes extracted.
Extracted Features DataFrame:
(1784, 148)
(1784,)


In [6]:
model = xgb.XGBClassifier(
    eval_metric='logloss'  # or 'auc' for binary classification
)

model.fit(
    train_X, train_y,
    verbose=100
)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [7]:
# evaluate model
test_preds = model.predict(test_X)

# calculate f1 score
from sklearn.metrics import f1_score, classification_report
f1 = f1_score(test_y, test_preds, average='weighted')
print("F1 Score:", f1)
print("Classification Report:")
print(classification_report(test_y, test_preds, target_names=classes))

F1 Score: 0.8412807376471715
Classification Report:
              precision    recall  f1-score   support

         egg       0.97      0.92      0.94        36
        good       0.85      0.90      0.87       138
         bad       0.70      0.64      0.67        77
    unusable       0.89      0.90      0.89       106

    accuracy                           0.84       357
   macro avg       0.85      0.84      0.84       357
weighted avg       0.84      0.84      0.84       357



In [8]:
# try a switch all class 2 (bad) to class 3 (unusable)
train_y = np.array([2 if y == 3 else y for y in train_y])
test_y = np.array([2 if y == 3 else y for y in test_y])

model = xgb.XGBClassifier(
    eval_metric='logloss'  # or 'auc' for binary classification
)
model.fit(
    train_X, train_y,
    verbose=100
)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [9]:
# evaluate model
test_preds = model.predict(test_X)

# calculate f1 score
from sklearn.metrics import f1_score, classification_report
f1 = f1_score(test_y, test_preds, average='weighted')
print("F1 Score:", f1)
print("Classification Report:")
print(classification_report(test_y, test_preds, target_names=['egg', 'good', 'bad/unusable']))

F1 Score: 0.8880120340737753
Classification Report:
              precision    recall  f1-score   support

         egg       0.97      0.94      0.96        36
        good       0.87      0.86      0.87       138
bad/unusable       0.89      0.90      0.89       183

    accuracy                           0.89       357
   macro avg       0.91      0.90      0.90       357
weighted avg       0.89      0.89      0.89       357

